# ORGANISM: Saccharomyces_cerevisiae
# Datasource: https://downloads.thebiogrid.org/BioGRID/Release-Archive/BIOGRID-3.5.182/

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df_original = pd.read_csv("data/BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-3.5.182.tab2.txt", sep='\t')

/home/ranap/anaconda3/envs/MLenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (1,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_original.head()

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,68770,851136,854020,31676,34272,YLR418C,YOL145C,CDC73,CTR9,L000002792,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
1,68771,854020,851136,34272,31676,YOL145C,YLR418C,CTR9,CDC73,CDP1|L000003477,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
2,68774,851136,854290,31676,34518,YLR418C,YOR123C,CDC73,LEO1,L000002792,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
3,68775,854290,851136,34518,31676,YOR123C,YLR418C,LEO1,CDC73,L000000936,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID
4,68778,851136,852582,31676,32973,YLR418C,YBR279W,CDC73,PAF1,L000002792,...,14759368,559292,559292,High Throughput,-,-,-,-,-,BIOGRID


In [4]:
df = df_original[["Official Symbol Interactor A", "Official Symbol Interactor B"]]

In [5]:
df.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,CDC73,CTR9
1,CTR9,CDC73
2,CDC73,LEO1
3,LEO1,CDC73
4,CDC73,PAF1


In [6]:
G = nx.from_pandas_edgelist(df, 'Official Symbol Interactor A', 'Official Symbol Interactor B', create_using=nx.DiGraph())

In [7]:
print(G.is_directed(), len(G.nodes), len(G.edges()))

True 7228 604680


In [8]:
# deleting self-loop edges
for n, nbrs in G.adj.items():
    if n in nbrs:
        G.remove_edge(n,n)

In [9]:
print(G.is_directed(), len(G.nodes), len(G.edges()))

True 7228 602919


### 3 nodes chain excluding common regulator 

In [10]:
def common_regulator(G, node1, node2):
    in_node1 = [a[0] for a in G.in_edges(node1)]
    in_node2 = [b[0] for b in G.in_edges(node2)]
    for i in in_node1:
        if i in in_node2:
            return True
        return False

In [11]:
common_regulator(G,'marA','putP')

In [12]:
node_set = set() # to save the node list
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if(node3 not in [node1, node2] and (node3 not in G.neighbors(node1)) and not(common_regulator(G, node1, node3))): 
                    print(node1, node2, node3)
                    node_set.add(node1); node_set.add(node2); node_set.add(node3);

CDC73 RPS31 CALCOCO2
POB3 PDC2 PFK1
CKB1 MSC3 LSP1
CEF1 PRM4 ICE2
PRP8 PDC2 PFK1
DSN1 RPL30 STO1
DSN1 EFR3 YPP1
DSN1 PDC2 PFK1
CHL4 PRM4 ICE2
MCM22 YAR010C ATG19
MTW1 PDC2 PFK1
SEC63 YBR219C OYE2
NAF1 HPM1 MAGEA11
CBF5 PDC2 PFK1
RPB3 RPS31 CALCOCO2
RPB2 RPS31 CALCOCO2
MMS21 PDC2 PFK1
MMS21 HPM1 MAGEA11
NSE1 PDC2 PFK1
NSE4 PFF1 SEC61
NSE4 YBR219C OYE2
SMC6 CPR4 SWP1
ESF1 PDC2 PFK1
NOP58 RPL30 STO1
ESA1 MSR1 PRE9
ESA1 DON1 ADY3
RVB1 HPM1 MAGEA11
RVB1 MNP1 BCP1
RVB1 RPS31 CALCOCO2
RVB2 RPL30 STO1
EAF1 COS9 MMP1
EAF5 PDC2 PFK1
YNG2 HPM1 MAGEA11
HSP60 PRM4 ICE2
HSP60 Bax BXI1
SSF1 RPL30 STO1
BRX1 HPM1 MAGEA11
DBP9 PRM4 ICE2
ERB1 PFF1 SEC61
NOP2 PRM4 ICE2
NOP7 RPL30 STO1
RPF2 RPL30 STO1
RPF2 RPS31 CALCOCO2
RPF2 YAR010C ATG19
RPL10 HPM1 MAGEA11
RPL16A RPL30 STO1
RPL38 PDC2 PFK1
RLP7 RPL30 STO1
RLP7 RPS31 CALCOCO2
RRP1 RPL30 STO1
KAP123 PRM4 ICE2
POP4 PDH1 KCS1
SNM1 YLR257W YLR345W
BRR2 PDC2 PFK1
BRR2 HPM1 MAGEA11
SMX3 HPM1 MAGEA11
NSE5 PDC2 PFK1
KRE29 PDC2 PFK1
KAR2 FYV6 RNR4
CDC42 PXL1 RHO1


SGV1 YAR010C ATG19
SGV1 HPM1 MAGEA11
SGV1 EFM2 EFT1
VPS5 HPM1 MAGEA11
TAF11 PDC2 PFK1
TAF3 PDC2 PFK1
TAF12 RPL30 STO1
TAF12 PFF1 SEC61
TAF13 ICS3 INO4
HEM13 YAR010C ATG19
HEM13 CPR4 SWP1
TAP42 PDC2 PFK1
TFA1 RPS31 CALCOCO2
TFC6 PDC2 PFK1
TFC6 RPS31 CALCOCO2
TFC8 PDC2 PFK1
THI3 PDC2 PFK1
TOA1 PDC2 PFK1
YOR1 YPL199C OXP1
BCY1 SFH5 PDR8
TPS1 YHL044W RPS28B
TRP2 PDC2 PFK1
TRP2 CPR4 SWP1
MNL1 RPL30 STO1
BRO1 PDC2 PFK1
BRO1 HPM1 MAGEA11
VAM6 MSC3 LSP1
VAM6 PXL1 RHO1
VAM6 RPL30 STO1
VAM6 RPS31 CALCOCO2
TLG2 HPM1 MAGEA11
FUN19 PDC2 PFK1
TCB3 PDC2 PFK1
SET1 STE13 SNX3
SET1 YOR238W LIN1
SET1 PDC2 PFK1
HEK2 STE13 SNX3
HEK2 YHL044W RPS28B
HEK2 PRM4 ICE2
HEK2 PDC2 PFK1
HEK2 MNP1 BCP1
HEK2 YAR010C ATG19
YBR238C DON1 ADY3
RIO2 RPL30 STO1
RIO2 RDN18-1 NOP9
RIO2 RPS31 CALCOCO2
GET3 STE13 SNX3
CAB5 YLR326W TCB3
PUF3 CPR4 SWP1
PUF3 MNP1 BCP1
PUF3 SFH5 PDR8
PUF3 RPS31 CALCOCO2
PUF3 PRM4 ICE2
PUF3 MSR1 PRE9
PUF3 FYV6 RNR4
NTE1 YHL044W RPS28B
YNK1 YLR326W TCB3
FAR11 PDC2 PFK1
TRM9 PDC2 PFK1
CIR2 SFH5 PDR8
C

OST2 PDC2 PFK1
OST4 STE13 SNX3
OST4 YPL199C OXP1
PET111 CPR4 SWP1
PEX3 HPM1 MAGEA11
RAP1 PDC2 PFK1
RAP1 ICS3 INO4
RAP1 EFM2 EFT1
RAP1 DIA3 RTG3
SEC15 SFH5 PDR8
SEC61 MF(ALPHA)1 STE2
TOM20 SFH5 PDR8
TOM20 SGTA SBH1
GLE1 PRM4 ICE2
STP22 HPM1 MAGEA11
YIP1 PDC2 PFK1
PRP16 CPR4 SWP1
PRP16 HPM1 MAGEA11
RPS19A RDN18-1 NOP9
MSS51 OLI1 UBQLN1
COX1 VAR1 SUV3
SCC2 RPS31 CALCOCO2
HAP1 PDC2 PFK1
SOM1 CPR4 SWP1
OXA1 VAR1 SUV3
OXA1 OLI1 UBQLN1
GLN3 HPM1 MAGEA11
GLN3 SFH5 PDR8
GPB1 UBE2D1 RSP5
YUH1 RPS31 CALCOCO2
ABF1 HPM1 MAGEA11
MUD2 PDC2 PFK1
HMT1 ECM12 BPH1
HMT1 RIB1 YPR172W
HMT1 DON1 ADY3
NTG2 SFH5 PDR8
FCP1 DCG1 LCB4
FCP1 DIA3 RTG3
SPT10 PDC2 PFK1
LRG1 PDC2 PFK1
AFI1 PDC2 PFK1
ERG25 MSC3 LSP1
PIG2 PFF1 SEC61
GAD1 CPR4 SWP1
YPS1 HPM1 MAGEA11
REC107 PDC2 PFK1
OTU1 RPS31 CALCOCO2
PET122 PDC2 PFK1
PEX11 HPM1 MAGEA11
PEX32 PDC2 PFK1
UBP2 RPS31 CALCOCO2
PUF4 MF(ALPHA)1 STE2
FIR1 HPM1 MAGEA11
SKY1 PDC2 PFK1
SLF1 MFA1 MFA2
SLF1 RPL30 STO1
SLF1 YLR257W YLR345W
SLF1 YLR326W TCB3
GFD2 HPM1 MAGEA11
YDL186W 

In [13]:
# H = G.subgraph(list(node_set))
# pos = nx.circular_layout(H)
# nx.draw(H, pos, cmap = plt.get_cmap('jet'), node_size = 2000)
# nx.draw_networkx_labels(H, pos)
# plt.show()

### 4 nodes chain

In [14]:
# pattern 2 search
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if (G.out_degree(node3)>1 or G.out_degree(node3)>1 or node3 in [node1, node2]):
                    continue
                else:
                    for node4 in G.neighbors(node3):
                        if(node4 not in [node1, node2, node3] and (node4 not in G.neighbors(node1)) and not(common_regulator(G, node1, node4))):
                            print(node1, node2, node3, node4)

### 5 nodes chain

In [15]:
# pattern 2 search
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if (G.out_degree(node3)>1 or G.out_degree(node3)>1 or node3 in [node1, node2]):
                    continue
                else:
                    for node4 in G.neighbors(node3):
                        if (G.out_degree(node4)>1 or G.out_degree(node4)>1 or node4 in [node1, node2, node3]):
                            continue
                        else:
                            for node5 in G.neighbors(node4):
                                if(node4 not in [node1, node2, node3, node4] and (node5 not in G.neighbors(node1)) and not(common_regulator(G, node1, node5))):
                                    print(node1, node2, node3, node4, node5)                            

Comment: No chain of length 5 found